In [16]:
import os 
import sys 
import builtins
from typing import List 

cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

from functools import cache 
import numpy as np 
import pandas as pd 
import altair as alt 
from palettable.tableau import Tableau_10
from IPython.display import clear_output
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../../.env')

from utils_notebook.utils import ddf, load_subgraph, remove_prefix
from utils_notebook.vega import (
    output_chart, 
    apply_css, 
    stack_order_expr, 
    wide_to_longwide,
    chart, 
)
from utils_notebook.queries import adjust_precision, QueryManager
from utils_notebook.testing import validate_season_series
from utils_notebook.css import css_tooltip_timeseries_multi_colored
from utils_notebook.queries import QueryManager
from utils_notebook.vega import output_chart

import warnings
warnings.filterwarnings('ignore')


In [2]:
sg: Subgrounds
bs: Subgraph
sg, bs = load_subgraph()

In [3]:
q = QueryManager(sg, bs) 

In [4]:
pool = bs.Query.metapoolOracles(first=100000, orderBy="season", orderDirection="asc")
df = sg.query_df(
    [
        pool.balanceA, 
        pool.balanceB, 
        pool.season, 
        pool.deltaB, 
        pool.timestamp, 
    ],
    pagination_strategy=ShallowStrategy
)
df = remove_prefix(df, "metapoolOracles_")
df.balanceA /= 10**6
df.balanceB /= 10**18
df.deltaB /= 10**6

In [5]:
# Reverse engineer pool reserves from quantites used in TWAP calculation. 
df['diff_a'] = (df.balanceA - df.balanceA.shift(1))
df['diff_b'] = (df.balanceB - df.balanceB.shift(1))
df['diff_timestamp'] = (df.timestamp - df.timestamp.shift(1))
df['reserves_3crv'] = df['diff_a'] / df.diff_timestamp
df['reserves_bean'] = df['diff_b'] / df.diff_timestamp
assert df.season.min() == 6076
df = df.iloc[1:,]
df.head()

balanceA              balanceB  season         deltaB  \
1  1823002386736.906738  2030942247135.286865    6077  685226.464263   
2  1869934826665.250244  2081743078278.517822    6078  694584.048577   
3  1917706461458.546631   2133501113442.06958    6079  701977.691800   
4  1965608261630.781006  2186128547794.177979    6080  803240.828322   
5  2013881826521.260254  2239501123147.967285    6081  849932.384158   

    timestamp              diff_a              diff_b  diff_timestamp  \
1  1659812464  46790827319.678223  50589125510.827393          3533.0   
2  1659816002  46932439928.343506  50800831143.230957          3538.0   
3  1659819600  47771634793.296387  51758035163.551758          3598.0   
4  1659823204  47901800172.234375  52627434352.108398          3604.0   
5  1659826833  48273564890.479248  53372575353.789307          3629.0   

     reserves_3crv    reserves_bean  
1   13243936.40523  14319027.883053  
2  13265245.881386  14358629.492151  
3  13277274.817481  14385223.780865  
4  13291287.506169  14602506.756967  
5   13302167.23353   14707240.38407

In [6]:
df_szns = q.query_seasons(extra_cols=['price'], where={"season_gte": 6074})[['season', 'price']]
df_szns = df_szns.rename(columns={"price": "price_bean"})
df_szns.head()

season  price_bean
0    6074    1.022000
1    6075    1.070000
2    6076    1.050748
3    6077    1.051615
4    6078    1.051964

In [7]:
df = df.merge(df_szns, how="left", on="season")
df.head()

balanceA              balanceB  season         deltaB  \
0  1823002386736.906738  2030942247135.286865    6077  685226.464263   
1  1869934826665.250244  2081743078278.517822    6078  694584.048577   
2  1917706461458.546631   2133501113442.06958    6079  701977.691800   
3  1965608261630.781006  2186128547794.177979    6080  803240.828322   
4  2013881826521.260254  2239501123147.967285    6081  849932.384158   

    timestamp              diff_a              diff_b  diff_timestamp  \
0  1659812464  46790827319.678223  50589125510.827393          3533.0   
1  1659816002  46932439928.343506  50800831143.230957          3538.0   
2  1659819600  47771634793.296387  51758035163.551758          3598.0   
3  1659823204  47901800172.234375  52627434352.108398          3604.0   
4  1659826833  48273564890.479248  53372575353.789307          3629.0   

     reserves_3crv    reserves_bean  price_bean  
0   13243936.40523  14319027.883053    1.051615  
1  13265245.881386  14358629.492151    1.051964  
2  13277274.817481  14385223.780865    1.052462  
3  13291287.506169  14602506.756967    1.062113  
4   13302167.23353   14707240.38407    1.063713

In [8]:
# Approximation of pool TVL in $, since we can't compute this exactly without the price of 3Crv 
df['pool_tvl_usd'] = 2 * df.price_bean * (df.reserves_bean - df.deltaB)
df['bean_fraction'] = (df.price_bean * df.reserves_bean) / df.pool_tvl_usd
df['3crv_fraction'] = 1 - df.bean_fraction
df.tail()

balanceA                balanceB  season        deltaB  \
2058   113429873739682.46875  110869454982437.921875    8135  -7660.251706   
2059  113479980920701.953125  110918421551236.890625    8136  -7657.047186   
2060  113530088101721.421875     110967388120035.875    8137  -7653.729622   
2061   113580195282740.90625   111016354688834.84375    8138  -7624.518383   
2062  113630324960715.421875   111065299270772.03125    8139 -13868.343960   

       timestamp              diff_a              diff_b  diff_timestamp  \
2058  1667221211   50107181019.46875  48966568798.984375          3600.0   
2059  1667224811  50107181019.484375   48966568798.96875          3600.0   
2060  1667228411   50107181019.46875  48966568798.984375          3600.0   
2061  1667232011  50107181019.484375   48966568798.96875          3600.0   
2062  1667235611  50129677974.515625    48944581937.1875          3600.0   

        reserves_3crv    reserves_bean  price_bean     pool_tvl_usd  \
2058  13918661.394297  13601824.666385    0.999449  27203972.183801   
2059  13918661.394301   13601824.66638    0.999449  27203965.778284   
2060  13918661.394297  13601824.666385    0.999450  27203986.365777   
2061  13918661.394301   13601824.66638    0.999452   27203982.41322   
2062  13924910.548477  13595717.204774    0.998930  27190046.584394   

     bean_fraction 3crv_fraction  
2058      0.499719      0.500281  
2059      0.499719      0.500281  
2060      0.499719      0.500281  
2061       0.49972       0.50028  
2062       0.49949       0.50051

In [9]:
df.timestamp = pd.to_datetime(df.timestamp, unit='s')
id_cols = ['timestamp']
value_cols = ['reserves_3crv', 'reserves_bean', 'deltaB', 'bean_fraction', '3crv_fraction', 'pool_tvl_usd']
df = df[id_cols + value_cols]
df = df.resample("D", on="timestamp").apply(lambda v: v.mean()).reset_index() 
df = wide_to_longwide(df, "timestamp", id_cols, value_cols)

In [10]:
df.tail()

timestamp       variable         value  reserves_3crv  reserves_bean  \
517 2022-10-31  reserves_bean  1.361055e+07   1.392391e+07   1.361055e+07   
518 2022-10-31         deltaB -5.837357e+03   1.392391e+07   1.361055e+07   
519 2022-10-31  bean_fraction  4.997858e-01   1.392391e+07   1.361055e+07   
520 2022-10-31  3crv_fraction  5.002142e-01   1.392391e+07   1.361055e+07   
521 2022-10-31   pool_tvl_usd  2.722310e+07   1.392391e+07   1.361055e+07   

          deltaB  bean_fraction  3crv_fraction  pool_tvl_usd  
517 -5837.357258       0.499786       0.500214  2.722310e+07  
518 -5837.357258       0.499786       0.500214  2.722310e+07  
519 -5837.357258       0.499786       0.500214  2.722310e+07  
520 -5837.357258       0.499786       0.500214  2.722310e+07  
521 -5837.357258       0.499786       0.500214  2.722310e+07

In [17]:
width = 450 
tooltip_metrics = [
    'reserves_3crv', 
    'reserves_bean',
    'bean_fraction', 
    '3crv_fraction',
    'deltaB', 
    'pool_tvl_usd', 
]
tooltip_formats = {
    "bean_fraction": ".1%", 
    "3crv_fraction": ".1%", 
    "pool_tvl_usd": "$,d"
}
colors = {
    'reserves_3crv': Tableau_10.hex_colors[0], 
    'reserves_bean': Tableau_10.hex_colors[1],
    'bean_fraction': Tableau_10.hex_colors[2], 
    '3crv_fraction': Tableau_10.hex_colors[3],
    'deltaB': Tableau_10.hex_colors[6], 
    'pool_tvl_usd': Tableau_10.hex_colors[8], 
}

chart_reserves, selection_nearest = chart(
    df, 
    "timestamp", 
    lmetrics=['reserves_3crv', 'reserves_bean'], 
    lstrategy='line', 
    l_yscales={"line": dict(domainMin=12*1e6)}, # TODO: don't hardcode this
    title="Bean:3Crv Pool Reserves", 
    yaxis_left_kwargs=dict(title="Reserves", format=".2~s"),    
    create_selection=True, 
    add_selection=True, 
    return_selection=True, 
    tooltip_metrics=tooltip_metrics, 
    tooltip_formats=tooltip_formats, 
    colors=colors, 
    width=width, 
)
chart_balance = chart(
    df, 
    "timestamp", 
    lmetrics=['bean_fraction', '3crv_fraction'], 
    lstrategy='line', 
    title="Bean:3Crv Pool Balance", 
    yaxis_left_kwargs=dict(title="Balance Ratio", format="%"),    
    l_yscales={"line": dict(domainMin=.45, domainMax=.55)}, # TODO: don't hardcode this
    selection_nearest=selection_nearest, 
    create_selection=False, 
    add_selection=True, 
    tooltip_metrics=tooltip_metrics, 
    tooltip_formats=tooltip_formats, 
    colors=colors, 
    width=width, 
)
chart_deltab = chart(
    df, 
    "timestamp", 
    lmetrics=['deltaB'], 
    lstrategy='line', 
    title="Bean:3Crv Pool DeltaB", 
    yaxis_left_kwargs=dict(title="DeltaB", format=".2~s"),    
    selection_nearest=selection_nearest, 
    create_selection=False, 
    add_selection=True, 
    tooltip_metrics=tooltip_metrics, 
    tooltip_formats=tooltip_formats,
    colors=colors, 
    width=width, 
)
chart_tvl = chart(
    df, 
    "timestamp", 
    lmetrics=['pool_tvl_usd'], 
    lstrategy='line', 
    title="Bean:3Crv Pool TVL", 
    yaxis_left_kwargs=dict(title="Pool TVL", format="$.2~s"),    
    selection_nearest=selection_nearest, 
    create_selection=False, 
    add_selection=True, 
    tooltip_metrics=tooltip_metrics, 
    tooltip_formats=tooltip_formats, 
    colors=colors, 
    width=width, 
)
c = (
    alt.vconcat(
        alt.hconcat(chart_reserves, chart_balance),
        alt.hconcat(chart_deltab, chart_tvl),
    )
    .resolve_scale(y="independent")
    .resolve_axis(y="independent")
)
css_lines = css_tooltip_timeseries_multi_colored(tooltip_metrics, colors)
css = "\n".join(css_lines)
# apply_css(css) 
apply_css("") 
c

alt.VConcatChart(...)

In [12]:
output_chart(c, css=css)

<IPython.core.display.JSON object>